# RNAseq data quality control
Notebook to examine the output of FastQC reports. Specifically, to check the species identity of overrepresented sequences in each sample.

In [1]:
from os import listdir
from os.path import splitext
from bs4 import BeautifulSoup
from io import StringIO
import pandas as pd
from Bio import Blast
from tqdm import tqdm
from collections import defaultdict

## Read in the overrepresented sequence tables
Each sample has a table of sequences in HTML format, so we can read them in to programmatically BLAST them later on. First, get all the files as `BeautifulSoup` objects:

In [2]:
qc_dir = '../data/20230131_mRNASeq_PE150/FastQC/'
qc_soups = {}
for f in listdir(qc_dir):
    if splitext(f)[1] == '.html':
        with open(qc_dir + f) as fp:
            soup = BeautifulSoup(fp, 'html.parser')
            qc_soups[splitext(f)[0]] = soup

Then, parse out the data we want:

In [3]:
def get_seq_table(soup):
    """
    Get the overrepresented sequences table from a FastQC html object.
    """
    overrep_table = soup.findAll('table')[1] # Gets the second table, assumes the presence of the first table with basic stats
    overrep_df = pd.read_html(StringIO(str(overrep_table)))[0]
    return overrep_df

In [4]:
qc_overrep_tables = {}
for samp, soup in qc_soups.items():
    try:
        qc_overrep_tables[samp] = get_seq_table(soup)
    except IndexError:
        print(f'Content of failed parse HTML {samp}:')
        print(soup)

Content of failed parse HTML 11_S10_L002_R2_001_fastqc:

Content of failed parse HTML 10_S9_L002_R2_001_fastqc:

Content of failed parse HTML 24_S23_L002_R1_001_fastqc:

Content of failed parse HTML 6_S5_L002_R1_001_fastqc:

Content of failed parse HTML 17_S16_L002_R1_001_fastqc:

Content of failed parse HTML 6_S5_L002_R2_001_fastqc:

Content of failed parse HTML 23_S22_L002_R2_001_fastqc:

Content of failed parse HTML 8_S7_L002_R2_001_fastqc:

Content of failed parse HTML 4_S3_L002_R1_001_fastqc:

Content of failed parse HTML 5_S4_L002_R1_001_fastqc:

Content of failed parse HTML 20_S19_L002_R1_001_fastqc:

Content of failed parse HTML 2_S1_L002_R1_001_fastqc:



For some reason, some of these FastQC files don't have any contents; I think these correspond to the documents I couldn't open previously, not sure what's happening there. For now going to carry on, but come back to this.

Snapshot of a table:

In [5]:
list(qc_overrep_tables.values())[0].head()

,Sequence,Count,Percentage,Possible Source
0,GGTCAATAAGGTAGGGATCATCAAAACACCAAACCATCCAATGTAA...,489823,2.824467,No Hit
1,GCGGTCAATAAGGTAGGGATCATCAAAACACCAAACCATCCAATGT...,477232,2.751864,No Hit
2,GTCAATAAGGTAGGGATCATCAAAACACCAAACCATCCAATGTAAA...,250043,1.441823,No Hit
3,GGTTAATAATATCAGCCCAAGTATTAATAACACGTCCTTGACTATC...,175930,1.014465,No Hit
4,GTTAATAATATCAGCCCAAGTATTAATAACACGTCCTTGACTATCA...,169422,0.976938,No Hit


## BLAST the sequences
We want to know if these overrepresented sequences come from Arabidopsis, and if so, what they are. We can do this by programmatically BLASTing all the sequences. First, lets put the sequences from each sample into a FASTA format to be able to BLAST them all at once:

In [7]:
def get_fasta(df):
    """
    Take the sequences from an overrepresented sequences table and put them in FASTA format to BLAST.
    """
    fasta_str = ''
    for i, row in df.iterrows():
        seq_name = f'>{i}\n'
        seq = row.Sequence
        if i < len(df) - 1:
            fasta_str += seq_name + seq + '\n'
        else:
            fasta_str += seq_name + seq
    return fasta_str

In [8]:
fastas = {k: get_fasta(v) for k, v in qc_overrep_tables.items()}

Now we BLAST and parse the results.

In [19]:
blast_results = {}
for samp, fasta in tqdm(fastas.items()):
    # Make a dict to be able to match sequence IDs to sequence, since biopython doesn't save the sequences
    fasta_list = fasta.split('\n')
    fasta_id_dict = {}
    for i, elt in enumerate(fasta_list): # Feels hacky but here we are
        if i % 2 == 0 and elt[0] == '>':
            fasta_id_dict[elt[1:]] = fasta_list[i+1]
    # Make an object to store the results in a way we can turn into the pandas df we want
    sample_results = defaultdict(list)
    # Perform the BLAST
    result_stream = Blast.qblast('blastn', 'nt', fasta)
    # Parse the output
    records = Blast.parse(result_stream)
    for record in records:
        seq_id = record.query.description
        sample_results['overrep_seq'].append(fasta_id_dict[seq_id])
        has_non_chloroplast = False
        all_hit_names = []
        all_hit_evals = []
        for hit in record:
            if 'chloroplast' not in hit.target.description.lower():
                has_non_chloroplast = True
            all_hit_names.append(hit.target.description)
            all_hit_evals.append(format(hit[0].annotations["evalue"], ".2g"))
        if 'arabidopsis thaliana' not in ' '.join(all_hit_names).lower():
            no_arabidopsis = True
        else:
            no_arabidopsis = False
        sample_results['has_non_chloroplast'].append(has_non_chloroplast)
        sample_results['no_arabidopsis'].append(no_arabidopsis)
        sample_results['all_hits'].append(' | '.join(all_hit_names))
        sample_results['all_hit_evals'].append(' | '.join(all_hit_evals))
    # Save the results to the overall results
    blast_results[samp] = sample_results

100%|██████████| 35/35 [21:22<00:00, 36.64s/it]


In [25]:
all_blast_results = defaultdict(list)
for samp, blast_res in blast_results.items():
    blast_res['sample'] = [samp.split('_')[0]] * len(blast_res['overrep_seq'])
    blast_res['strand'] = [samp.split('_')[3]] * len(blast_res['overrep_seq'])
    for k, v in blast_res.items():
        all_blast_results[k].extend(v)

In [26]:
all_blast_res_df = pd.DataFrame.from_dict(all_blast_results).set_index(['sample', 'strand'])
all_blast_res_df.head()

overrep_seq  \
sample strand                                                      
23     R1      GGTCAATAAGGTAGGGATCATCAAAACACCAAACCATCCAATGTAA...   
       R1      GCGGTCAATAAGGTAGGGATCATCAAAACACCAAACCATCCAATGT...   
       R1      GTCAATAAGGTAGGGATCATCAAAACACCAAACCATCCAATGTAAA...   
       R1      GGTTAATAATATCAGCCCAAGTATTAATAACACGTCCTTGACTATC...   
       R1      GTTAATAATATCAGCCCAAGTATTAATAACACGTCCTTGACTATCA...   

               has_non_chloroplast  no_arabidopsis  \
sample strand                                        
23     R1                     True            True   
       R1                     True            True   
       R1                     True            True   
       R1                     True           False   
       R1                     True           False   

                                                        all_hits  \
sample strand                                                      
23     R1      Hydrangea petiolaris voucher YNUHHYD015 chloro...   
       R1      Hydrangea petiolaris voucher YNUHHYD015 chloro...   
       R1      Hydrangea petiolaris voucher YNUHHYD015 chloro...   
       R1      Cardamine bulbifera chloroplast, complete geno...   
       R1      Cardamine bulbifera chloroplast, complete geno...   

                                                   all_hit_evals  
sample strand                                                     
23     R1      3.5e-15 | 3.5e-15 | 3.5e-15 | 3.5e-15 | 3.5e-1...  
       R1      3.5e-15 | 3.5e-15 | 3.5e-15 | 3.5e-15 | 3.5e-1...  
       R1      3.5e-15 | 3.5e-15 | 3.5e-15 | 3.5e-15 | 3.5e-1...  
       R1      3.5e-15 | 3.5e-15 | 3.5e-15 | 3.5e-15 | 3.5e-1...  
       R1      3.5e-15 | 3.5e-15 | 3.5e-15 | 3.5e-15 | 3.5e-1...

In [27]:
all_blast_res_df.to_csv('../data/20230131_mRNASeq_PE150/blast_results_overrep_sequences.csv')

Read BLAST results back in to skip above code:

In [28]:
all_blast_res_df = pd.read_csv('../data/20230131_mRNASeq_PE150/blast_results_overrep_sequences.csv').set_index(['sample', 'strand']).sort_index(level='sample')
all_blast_res_df.head()

overrep_seq  \
sample strand                                                      
2      R2      GAGAATTTGTGCGCTTGGGAGTCCCTGATTATTAAATAAACCAAGG...   
       R2      GGAATCTCTGGTACTTTCAACTTTATGATTGTATTCCAGGCTGAGC...   
       R2      AGAGAATTTGTGCGCTTGGGAGTCCCTGATTATTAAATAAACCAAG...   
       R2      ATTTGGTTTACTGCTTTAGGTATTAGTACTATGGCTTTCAACCTAA...   
       R2      GGTACTTTCAACTTTATGATTGTATTCCAGGCTGAGCACAACATTC...   

               has_non_chloroplast  no_arabidopsis  \
sample strand                                        
2      R2                     True           False   
       R2                     True            True   
       R2                     True           False   
       R2                     True           False   
       R2                     True           False   

                                                        all_hits  \
sample strand                                                      
2      R2      Arabidopsis thaliana cultivar Tsu0 chloroplast...   
       R2      Oxybasis glauca isolate ADIFF_NF_020-psbA chlo...   
       R2      Arabidopsis thaliana cultivar Tsu0 chloroplast...   
       R2      Apinagia guyanensis voucher (R) C.P. Bove and ...   
       R2      Cardamine bulbifera chloroplast, complete geno...   

                                                   all_hit_evals  
sample strand                                                     
2      R2      3.5e-15 | 3.5e-15 | 3.5e-15 | 3.5e-15 | 3.5e-1...  
       R2      3.5e-15 | 3.5e-15 | 3.5e-15 | 3.5e-15 | 3.5e-1...  
       R2      3.5e-15 | 3.5e-15 | 3.5e-15 | 3.5e-15 | 3.5e-1...  
       R2      3.5e-15 | 3.5e-15 | 3.5e-15 | 3.5e-15 | 3.5e-1...  
       R2      3.5e-15 | 3.5e-15 | 3.5e-15 | 3.5e-15 | 3.5e-1...

## Explore results
Now that we have the top BLAST result for all of the overrepresented sequences, let's take a look at what they are. For each sample, the first sequence is the most overrepresented, so we can start by looking at those, especially since we know that for a lot of samples, the first sequences is much much more overrepresented than all the rest.

In [38]:
all_blast_res_df.groupby(['sample', 'strand'])['no_arabidopsis'].any()

sample  strand
2       R2        True
3       R1        True
        R2        True
4       R2        True
5       R2        True
7       R1        True
        R2        True
8       R1        True
9       R1        True
        R2        True
10      R1        True
11      R1        True
12      R1        True
        R2        True
13      R1        True
        R2        True
14      R1        True
        R2        True
15      R1        True
        R2        True
16      R2        True
17      R2        True
18      R1        True
        R2        True
19      R1        True
        R2        True
20      R2        True
21      R1        True
        R2        True
22      R1        True
        R2        True
23      R1        True
24      R2        True
25      R1        True
        R2        True
Name: no_arabidopsis, dtype: bool

In [39]:
all_blast_res_df.groupby(['sample', 'strand'])['no_arabidopsis'].all()

sample  strand
2       R2        False
3       R1        False
        R2        False
4       R2        False
5       R2        False
7       R1        False
        R2        False
8       R1        False
9       R1        False
        R2        False
10      R1        False
11      R1        False
12      R1        False
        R2        False
13      R1        False
        R2        False
14      R1        False
        R2        False
15      R1        False
        R2        False
16      R2        False
17      R2        False
18      R1        False
        R2        False
19      R1        False
        R2        False
20      R2        False
21      R1        False
        R2        False
22      R1        False
        R2        False
23      R1        False
24      R2        False
25      R1        False
        R2        False
Name: no_arabidopsis, dtype: bool

In [40]:
all_blast_res_df.groupby(['sample', 'strand'])['has_non_chloroplast'].any()

sample  strand
2       R2        True
3       R1        True
        R2        True
4       R2        True
5       R2        True
7       R1        True
        R2        True
8       R1        True
9       R1        True
        R2        True
10      R1        True
11      R1        True
12      R1        True
        R2        True
13      R1        True
        R2        True
14      R1        True
        R2        True
15      R1        True
        R2        True
16      R2        True
17      R2        True
18      R1        True
        R2        True
19      R1        True
        R2        True
20      R2        True
21      R1        True
        R2        True
22      R1        True
        R2        True
23      R1        True
24      R2        True
25      R1        True
        R2        True
Name: has_non_chloroplast, dtype: bool

In [41]:
all_blast_res_df.groupby(['sample', 'strand'])['has_non_chloroplast'].all()

sample  strand
2       R2        False
3       R1        False
        R2        False
4       R2        False
5       R2        False
7       R1        False
        R2        False
8       R1        False
9       R1        False
        R2        False
10      R1        False
11      R1        False
12      R1        False
        R2        False
13      R1        False
        R2        False
14      R1        False
        R2        False
15      R1        False
        R2        False
16      R2        False
17      R2        False
18      R1        False
        R2        False
19      R1        False
        R2        False
20      R2        False
21      R1        False
        R2        False
22      R1        False
        R2        False
23      R1        False
24      R2        False
25      R1        False
        R2        False
Name: has_non_chloroplast, dtype: bool

My takeaway from this is:
* There are some sequences that are not chloroplast-identified in all samples
* There are some sequences that don't match to Arabidopsis in all samples
* Directionality of strand doesn't matter

Jenny pointed out that Chloroplast RNA only has polyA tails if it's marked for degredation, so if we used polyA enrichment, then these are only RNA marked for degredation.